<a href="https://colab.research.google.com/github/dnyanda12/assignments/blob/main/narmadaGIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap

In [2]:
import ee

In [4]:
ee.Authenticate()

In [5]:
ee.Initialize(project= "ee-dnyandagawathe24")

In [6]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
India = ee.FeatureCollection("projects/ee-dnyandagawathe24/assets/India_state")
m.addLayer(India, {}, "India")
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [8]:
Narmada = ee.FeatureCollection("projects/ee-dnyandagawathe24/assets/NARMADA")
m.addLayer(Narmada, {}, "Narmada")
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [12]:
Narmada_Buffer = 50000
Narmada_buffer = Narmada.map(lambda f: f.buffer(Narmada_Buffer))
m.addLayer(Narmada_buffer, {}, "Narmada_buffer")
m

Map(bottom=300.0, center=[85.0511287798066, 246.09375000000003], controls=(WidgetControl(options=['position', …

In [13]:
NDWI = ee.ImageCollection("LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI").select('NDWI')

In [14]:
NDWI_Narmada = NDWI.map(lambda img: img.set('doy',ee.Date(img.get('system:time_start')).getRelative('day', 'year')));
distinctDOY = NDWI_Narmada.filterDate('2022-01-01', '2023-12-31');

In [15]:
filter = ee.Filter.equals(leftField='doy', rightField ='doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDWI_Narmada, filter))

In [16]:
def add_date_from(img):
  doy = ee.Number(img.get('doy'))

In [17]:
composite = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy',ee.Number(img.get('doy'))))

In [18]:
Vis_para = {
  'min': 0.0,
  'max': 0.1,
  'palette': [
    '0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff'
  ],
};

In [19]:
rgbvis= composite.map(lambda img: img.visualize(bands = ['NDWI_mean'],**Vis_para).clip(Narmada_buffer))

In [20]:
gifParams = {
  'region': Narmada_buffer.geometry(),
  'dimensions': 600,
  'crs': 'EPSG:4326',
  'framesPerSecond': 80,
  'format':'gif',
  'addDate': 'yyyy-mm-dd'
}

In [21]:
print(rgbvis.getVideoThumbURL(gifParams))

https://earthengine.googleapis.com/v1/projects/ee-dnyandagawathe24/videoThumbnails/411d1cb8cea0f2b4e5ecd1832323cc99-e53f13b754d3db4c16a8edf9ff4216a9:getPixels
